## Project Purpose
Developing a Sentiment Analysis project where the ups and downs of Twitter sentiment are tracked for a pet related company.

The motivation for this analysis is to help a company see what topics or other factors might correlate with better or worse sentiment towards a company

Most of the code for pulling the Twitter data was gleaned from Samantha Jone's article on Tweepy: https://www.linkedin.com/pulse/extracting-tweet-information-tweepy-beautifulsoup-samantha-jones/###

Samantha Jones: https://www.linkedin.com/in/samanthaaldene/ 

The Tweepy site is self is here: https://www.tweepy.org/

## Import Packages

In [1]:
import os
import pandas as pd
import numpy as np
import tweepy
from tweepy import OAuthHandler
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup as bs
from urllib.parse import urljoin, urlparse
from requests_oauthlib import OAuth1
from cleantext import clean

In [172]:
#Handle Twitter API authentication; produces a json file

consumer_key = "s2RQHHhTBhg7rmGP2r8K9bKmZ"
consumer_secret = "BNspDCtvyIIWSfSaZ3sWVDE82JSqVAoel4oxiD7rjiKhHCq1rZ"
access_token = "4243232613-GfIElLNDpFCVWUlMqMU6ZGSah9uVfJZJ6W1AeCR"
access_token_secret = "rURKXxsocX8ivZVEVugOG0Y8ti9VTIS2nzUAVztCWSI9s"
 
auth = tweepy.OAuth1UserHandler(
   consumer_key, consumer_secret, access_token, access_token_secret
)

api = tweepy.API(auth)

###testing out the api connection
public_tweets = api.home_timeline()
for tweet in public_tweets:
    print(tweet.text)

RT @nicolaiarips: I literally keep pitching about these girls and nobody BITES. I’m sick of it. I’m obsessed with them I want to write abou…
RT @tetisheri: Recruiting for a Study: Are you a UX professional? We at UC Santa Cruz are doing a small remote research study to understand…
RT @Zochrot: Here's why we encourage cities across the world to follow Barcelona historical move &amp; end their twinning agreements with #Isra…
RT @BrightonBDS: #Barcelona has cut twinning ties with Tel Aviv.
Maybe these UK councils can now follow suit:
@BCPCouncil &gt; Netanya.
@Newca…
RT @steketeh: Barcelona Mayor announces suspension of the city's relations with Israel incl twinning with Tel Aviv. In the face of aparthei…
I have seen more shit (literally) on the sidewalk in 6 months living here than in 10 years in New York
Wild how cancelling some topic on my syllabus in order to make extra time for us to practice what we've already bee… https://t.co/au1LPVNKGu
When I see a door-to-door salesman walking f

In [173]:
userID = "Purina"
tweets_purina = api.user_timeline(screen_name=userID, 
                           count=200,
                           include_rts = False)
print("Number of Tweets Extracted: {}.\n".format(len(tweets)))

Number of Tweets Extracted: 200.



In [174]:
userID = "DiamondPetFood"
tweets_diamond = api.user_timeline(screen_name=userID, 
                           count=200,
                           include_rts = False)
print("Number of Tweets Extracted: {}.\n".format(len(tweets)))

Number of Tweets Extracted: 200.



In [189]:
userID = "wellnesspetfood"
tweets_wellness = api.user_timeline(screen_name=userID, 
                           count=200,
                           include_rts = False)
print("Number of Tweets Extracted: {}.\n".format(len(tweets)))

Number of Tweets Extracted: 200.



In [176]:
userID = "bluebuffalo"
tweets_bluebuffalo = api.user_timeline(screen_name=userID, 
                           count=200,
                           include_rts = False)
print("Number of Tweets Extracted: {}.\n".format(len(tweets)))

Number of Tweets Extracted: 200.



In [177]:
#simple overview to make sure that it pulls correctly...And it looks good
for tweet in tweets_bluebuffalo[:5]:
    print(tweet.text)

@AlonsoOg Thank you for making us aware of this. We’re sorry to see this, and we are committed to making this right… https://t.co/P6qJQOGArk
@MonicaEli04 Max, ofc u qualify 😍😍
@murphy_the_gr8 Oughe !
@GingerLoafTigs @sarge_the_kitty @benny_the_kitty yeah. . . Friends
@Skullcattery22 Lizzy, patiently waiting....😍


In [192]:
#create dataframe to house data.
purina_df = pd.DataFrame(data=[[len(tweet.text), tweet.text, tweet.created_at, tweet.user.screen_name] for tweet in tweets_purina], 
                         columns = ['Tweet_Length', 'Tweet_Text', 'Tweet_Date', 'UserName'])
purina_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   Tweet_Length  199 non-null    int64              
 1   Tweet_Text    199 non-null    object             
 2   Tweet_Date    199 non-null    datetime64[ns, UTC]
 3   UserName      199 non-null    object             
dtypes: datetime64[ns, UTC](1), int64(1), object(2)
memory usage: 6.3+ KB


In [190]:
wellness_df = pd.DataFrame(data=[[len(tweet.text), tweet.text, tweet.created_at, tweet.user.screen_name] for tweet in tweets_wellness], 
                           columns = ['Tweet_Length', 'Tweet_Text', 'Tweet_Date', 'UserName'])
wellness_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   Tweet_Length  199 non-null    int64              
 1   Tweet_Text    199 non-null    object             
 2   Tweet_Date    199 non-null    datetime64[ns, UTC]
 3   UserName      199 non-null    object             
dtypes: datetime64[ns, UTC](1), int64(1), object(2)
memory usage: 6.3+ KB


In [191]:
bluebuffalo_df = pd.DataFrame(data=[[len(tweet.text), tweet.text, tweet.created_at, tweet.user.screen_name] for tweet in tweets_bluebuffalo], 
                              columns = ['Tweet_Length', 'Tweet_Text', 'Tweet_Date', 'UserName'])
bluebuffalo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   Tweet_Length  200 non-null    int64              
 1   Tweet_Text    200 non-null    object             
 2   Tweet_Date    200 non-null    datetime64[ns, UTC]
 3   UserName      200 non-null    object             
dtypes: datetime64[ns, UTC](1), int64(1), object(2)
memory usage: 6.4+ KB


In [186]:
diamond_df = pd.DataFrame(data=[[len(tweet.text), tweet.text, tweet.created_at, tweet.user.screen_name] for tweet in tweets_diamond], 
                          columns = ['Tweet_Length', 'Tweet_Text', 'Tweet_Date', 'UserName'])
diamond_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   Tweet_Length  200 non-null    int64              
 1   Tweet_Text    200 non-null    object             
 2   Tweet_Date    200 non-null    datetime64[ns, UTC]
 3   UserName      200 non-null    object             
dtypes: datetime64[ns, UTC](1), int64(1), object(2)
memory usage: 6.4+ KB


With four different dataframes, the question is how these four companies compare in terms of sentiment analysis.

Let's first take a look at combining and cleaning these datasets, preparing them for NLP analysis:

Will I want a separate column for those accounts tagged?
How about emoji usage? Should I count those and the accounts tagged and have them as intergers to see if I can run an ML model on the percent sentiment? Can you do an ML model on percent or do I need to convert to 1 for over 50% and 0 for under?

In [193]:
frames = [purina_df, wellness_df, bluebuffalo_df, diamond_df]
df = pd.concat(frames)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 798 entries, 0 to 199
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   Tweet_Length  798 non-null    int64              
 1   Tweet_Text    798 non-null    object             
 2   Tweet_Date    798 non-null    datetime64[ns, UTC]
 3   UserName      798 non-null    object             
dtypes: datetime64[ns, UTC](1), int64(1), object(2)
memory usage: 31.2+ KB


In [194]:
df['UserName'].value_counts()

bluebuffalo        200
DiamondPetFood     200
Purina             199
wellnesspetfood    199
Name: UserName, dtype: int64

In [92]:
pd.set_option('display.max_colwidth', None)

In [93]:
#Let's take a look at some sample tweets to see what is possible here:
df['Tweet_Text'][:10]
### Might be useful to count how many times each tweet responds or @ some other user; Also that "Please" that Purina keeps
### throwing out there . . . yikes. That kind of discourse marker is something to watch for.

0                      Daisy would like you to tell her she's a good girl. Don't worry, she'll wait. ☺️ \n\n📸: Spencer C. https://t.co/dUNZnvlVCI
1                                                     Reply and let us know what your pet's favorite love language is! ❤️ https://t.co/aL9AeCFyXL
2                                                 @TerriHorat Hi Terri! That's PAWsome! Thanks so much for being a loyal Diamond Pet Foods fan! 🤩
3                                                           @GreenMo72429984 Raise the woof! Thank you for being a loyal Diamond Pet Foods fan! 🐾
4                                       @AvonStaceys Hi Stacey! We’re so fur-tunate! 🐾 Thanks to you and your pup for choosing Diamond Pet Foods!
5                                                           POV: The dentist just told you to "open wide."\n\n📸: Janna C. https://t.co/Oy9cvOkdCp
6    If you find that your pet is sleeping the day away even more than usual, there might be a reason. Check out our blo… ht

In [ ]:
df_trim = df
df_trim = df['Tweet_Text'].str.split("http", n=1, expand=True)
df_trim[1] = 'http' + df_trim[1].astype(str)
df_trim.columns = ["Full_Tweets", "Tweet_Links"]
df_trim["Tweet_Links"]=df_trim['Tweet_Links'].str.split().str[0]

Separating out mentions in order to count them

In [196]:
def find_at_word(text):
    word=re.findall(r'(?<=@)\w+',text)
    return " ".join(word)

df_trim['at_word']=df_trim['Full_Tweets'].apply(lambda x: find_at_word(x))

In [197]:
df_trim.head(3)

,Full_Tweets,Tweet_Links,at_word
0,@lhartness Thank you for reaching out. We’re sorry that we are experiencing some technical difficulties and our ded…,https://t.co/k5ncW6PGIU,lhartness
1,@fireshadowed Thank you for reaching out. We’re sorry that we are experiencing some technical difficulties and our…,https://t.co/tABQff7epw,fireshadowed
2,"@BonnieblueBlue Thank you for reaching out to us. When you have a chance, please send us a private message with th…",https://t.co/GMsyUvy5Oq,BonnieblueBlue


In [209]:
frames = [df, df_trim]
df_new = pd.concat(frames, axis=1)
df_new = df_new.drop(columns = 'Tweet_Text')
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 798 entries, 0 to 199
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   Tweet_Length  798 non-null    int64              
 1   Tweet_Date    798 non-null    datetime64[ns, UTC]
 2   UserName      798 non-null    object             
 3   Full_Tweets   798 non-null    object             
 4   Tweet_Links   798 non-null    object             
 5   at_word       798 non-null    object             
dtypes: datetime64[ns, UTC](1), int64(1), object(4)
memory usage: 43.6+ KB


In [210]:
df_new['UserName'].value_counts()

bluebuffalo        200
DiamondPetFood     200
Purina             199
wellnesspetfood    199
Name: UserName, dtype: int64

In [211]:
df_new['Full_Tweets'] = df_new['Full_Tweets'].apply(str.lower)

Expanding contractions here using: https://towardsdatascience.com/a-practitioners-guide-to-natural-language-processing-part-i-processing-understanding-text-9f4abfd13e72

In [201]:
# -*- coding: utf-8 -*-
"""
Created on Mon Aug 01 01:11:02 2016

@author: DIP
"""

CONTRACTION_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [202]:
# Pulling from DJ Sarkar, cited above

def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

expand_contractions("Y'all can't expand contractions I'd think")
# So that works nicely, now to apply it to the full tweets

'You all cannot expand contractions I would think'

In [212]:
df_new['Full_Tweets'] = df_new['Full_Tweets'].apply(lambda x: expand_contractions(x))

In [213]:
df_new['Full_Tweets'].tail(3)

197    "sorry, i have plans."\n\nthe plans:\n- petting my dog\n- playing with my dog\n- cuddling with my dog
198                               hermes loves dates at the dog park! 🐕 🌳 🍂 anybody else?\n\n📸: corrinne k. 
199                                                "where did all of these cardboard boxes come from?!" 😸 📦 
Name: Full_Tweets, dtype: object

Removing emoji's using clean from clean-text

In [205]:
#provide string with emojis
text = "This sample text contains laughing emojis 😀 😃 😄 😁 😆 😅 😂 🤣"

#print text after removing the emojis from it
print(clean(text, no_emoji=True))


this sample text contains laughing emojis


In [214]:
df_new['Full_Tweets'] = df_new['Full_Tweets'].apply(lambda x: (clean(x, no_emoji=True)))

In [217]:
#Removing the mentions themselves from the Full_Tweets column; already separated off

df_new['Full_Tweets'] = df_new['Full_Tweets'].apply(lambda x: re.sub("@[A-Za-z0-9_]+","", x))

In [218]:
df_new.head(3)

,Tweet_Length,Tweet_Date,UserName,Full_Tweets,Tweet_Links,at_word
0,140,2023-02-09 19:42:43+00:00,Purina,thank you for reaching out. we're sorry that we are experiencing some technical difficulties and our ded,https://t.co/k5ncW6PGIU,lhartness
1,139,2023-02-09 19:41:43+00:00,Purina,thank you for reaching out. we're sorry that we are experiencing some technical difficulties and our,https://t.co/tABQff7epw,fireshadowed
2,140,2023-02-09 16:09:53+00:00,Purina,"thank you for reaching out to us. when you have a chance, please send us a private message with th",https://t.co/GMsyUvy5Oq,BonnieblueBlue


In [219]:
#Now remove punctuation
df_new['Full_Tweets'] = df_new['Full_Tweets'].apply(lambda x: re.sub("[^a-z0-9]"," ", x))

In [220]:
df_new.head()

,Tweet_Length,Tweet_Date,UserName,Full_Tweets,Tweet_Links,at_word
0,140,2023-02-09 19:42:43+00:00,Purina,thank you for reaching out we re sorry that we are experiencing some technical difficulties and our ded,https://t.co/k5ncW6PGIU,lhartness
1,139,2023-02-09 19:41:43+00:00,Purina,thank you for reaching out we re sorry that we are experiencing some technical difficulties and our,https://t.co/tABQff7epw,fireshadowed
2,140,2023-02-09 16:09:53+00:00,Purina,thank you for reaching out to us when you have a chance please send us a private message with th,https://t.co/GMsyUvy5Oq,BonnieblueBlue
3,144,2023-02-09 05:00:15+00:00,Purina,thanks for reaching out about alpo come get it we are sorry to share that while most sizes have been,https://t.co/VsPUdZPWfP,waffletower
4,94,2023-02-08 20:44:37+00:00,Purina,we can t keep our tails from wagging in anticipation of the purinapetparade,httpNone,OpenDoorAnimals


In [221]:
df_new['at_word'].value_counts()


                   254
swagmastarpaul      11
BengalsMeggy         9
reallylittlecat      5
fukkinnobody         5
                  ... 
DonaldBestCA         1
StxxApparel          1
SAMUELUSA2           1
EmmaPrive            1
RalfusJ              1
Name: at_word, Length: 443, dtype: int64

In [222]:
def count_at(mention):
    count = 0
    for item in mention.split():
        count += 1
    return count

test = "kirk kate cleo"

print(count_at(test))

3


In [223]:
df_new['Mention_Count'] = df_new['at_word'].apply(count_at)

In [224]:
df_new.tail(3)

,Tweet_Length,Tweet_Date,UserName,Full_Tweets,Tweet_Links,at_word,Mention_Count
197,96,2022-11-29 19:48:02+00:00,DiamondPetFood,sorry i have plans the plans petting my dog playing with my dog cuddling with my dog,httpNone,,0
198,95,2022-11-29 19:47:06+00:00,DiamondPetFood,hermes loves dates at the dog park anybody else corrinne k,https://t.co/DIAAi3VbUX,,0
199,80,2022-11-28 21:45:06+00:00,DiamondPetFood,where did all of these cardboard boxes come from,https://t.co/Kr8AlRG8GD,,0


In [225]:
df_new['Mention_Count'].value_counts()
# so unbalanced but a difference amongst them

1    524
0    254
2     19
3      1
Name: Mention_Count, dtype: int64

Save the crafted dataframe as a CSV for the EDA

In [226]:
df_new.to_csv('SA_data_clean.csv', index=False)